In [ ]:
import json
import csv
import os
import copy
import random
import torch
from datasets import load_from_disk
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import sys

sys.path.append('..')
from tools.TextProcessing import build_word_tree, process_keywords, nlp, clean_text
from tools.BasicUtils import my_write, my_csv_read, my_read, my_json_read
from tools.OpenIEUtils import processed_file_reader, openie_my_map

In [ ]:
# Collect keywords from terms-cs-cfl-epoch200.txt
stable_kw = []
unstable_kw = []
r = my_csv_read('../data/raw_data/terms-cs-cfl-epoch200.txt', delimiter='\t')
candidate_kw_list = [item[0] for item in r if float(item[1]) > 0.1]
stable_kw, unstable_kw = process_keywords(candidate_kw_list)
# Save keywords
if not os.path.exists('data'):
    os.mkdir('data')
my_write('data/keyword.txt', stable_kw)
# Generate word tree (25 seconds)
build_word_tree('data/keyword.txt', 'data/wordtree.json', 'data/entity.txt')

In [ ]:
# Go to py folder and run followings in the backend 
# "python gen_co_occur.py ../joint_score_func/data/wordtree.json ../data/corpus/small_sent.txt ../joint_score_func/data/co_occur.txt"
# "python gen_occur.py ../joint_score_func/data/keyword.txt ../joint_score_func/data/co_occur.txt ../joint_score_func/data/occur.json"

<h2> Generate dataset using Wikidata knowledge graph

In [ ]:
# Knowledge Graph filtering

# Load known cs keywords
kw_set = set(my_read('data/keyword.txt'))
# Get potential cs entity id
eid_set = set([eid for eid, ent in my_csv_read('../data/raw_data/wikidata/entity_names.txt', delimiter='\t') if ent.lower() in kw_set])
# Get the subgraph that both entities are potential cs keywords
kg_cs_triples = [(eid1, eid2, rid) for eid1, eid2, rid in my_csv_read('../data/raw_data/wikidata/triples.txt', delimiter=' ') if eid1 in eid_set and eid2 in eid_set]
# Get cs entities and relations from subgraph
cs_eid_set = set()
cs_rid_set = set()
for eid1, eid2, rid in kg_cs_triples:
    cs_eid_set.update((eid1, eid2))
    cs_rid_set.add(rid)
# Map id to text
eid2ent_dict = {eid:ent.lower() for eid, ent in my_csv_read('../data/raw_data/wikidata/entity_names.txt', delimiter='\t') if eid in cs_eid_set}
rid2rel_dict = {rid:rel.lower() for rid, rel in my_csv_read('../data/raw_data/wikidata/relation_names.txt', delimiter='\t') if rid in cs_rid_set}
# Save files
json.dump(eid2ent_dict, open('data/eid2ent.json', 'w'))
json.dump(rid2rel_dict, open('data/rid2rel.json', 'w'))
csv.writer(open('data/kg_cs_triples.csv', 'w')).writerows(kg_cs_triples)

In [ ]:
# Filter out pairs that have little co-occurance
occur_dict = my_json_read('data/occur.json')
occur_dict = {k:set(v) for k, v in occur_dict.items()}
acceptable_triple_data = []
for eid1, eid2, rid in kg_cs_triples:
    ent1, ent2 = eid2ent_dict[eid1], eid2ent_dict[eid2]
    if len(occur_dict[ent1] & occur_dict[ent2]) > 10 and rid in rid2rel_dict:
        acceptable_triple_data.append((ent1, ent2, rid2rel_dict[rid]))
csv.writer(open('data/datasets.csv', 'w')).writerows(acceptable_triple_data)

<h2> Generate dataset using OpenIE training data

In [ ]:
openie_triples = json.load(open('../data/corpus/openie_triples.json'))

In [ ]:
filtered_triples = copy.deepcopy(openie_triples)
for item in filtered_triples:
    item['triples'] = [tri for tri in item['triples'] if tri[0] >= 0.9]

In [ ]:
filtered_triples[3]

In [ ]:
openie_triples[3]

<h3> Ollie extraction on arxiv corpus

In [ ]:
low_score_list = processed_file_reader('../openie/ollie_test/small_processed_low.txt')
high_score_list = processed_file_reader('../openie/ollie_test/small_processed_high.txt')
all_score_list = processed_file_reader('../openie/ollie_test/small_processed_all.txt')

In [ ]:
low_score_triple = [item for item in low_score_list if len(item) > 1]
high_score_triple = [item for item in high_score_list if len(item) > 1]
no_extraction = [item for item in all_score_list if len(item) == 1]
print(len(low_score_triple))
print(len(high_score_triple))
print(len(no_extraction))

In [ ]:
# Generate pos dataset
pos_dataset = []
for item in high_score_triple:
    sent = clean_text(item[0])
    for triple in item[1:]:
        ent1, rel, ent2 = triple.split(';')
        ent1, rel, ent2 = clean_text(ent1), clean_text(rel), clean_text(ent2)
        pos_dataset.append({'label' : 1, 'ent1' : ent1, 'rel' : rel, 'ent2' : ent2, 'sent' : sent})

In [ ]:
# Generate neg dataset from low score triples
neg_dataset_1 = []
for item in low_score_triple:
    sent = clean_text(item[0])
    for triple in item[1:]:
        ent1, rel, ent2 = triple.split(';')
        ent1, rel, ent2 = clean_text(ent1), clean_text(rel), clean_text(ent2)
        neg_dataset_1.append({'label' : 0, 'ent1' : ent1, 'rel' : rel, 'ent2' : ent2, 'sent' : sent})

In [ ]:
# Generate neg dataset from no extraction sentences with noun chunks
neg_dataset_2 = []
for item in no_extraction:
    sent = clean_text(item[0])
    noun_chunks = list(nlp(sent).noun_chunks)
    if len(noun_chunks) <= 1:
        continue
    ents = random.sample(noun_chunks, 2)
    neg_dataset_2.append({'label' : 0, 'ent1' : str(ents[0]), 'rel' : '_', 'ent2' : str(ents[1]), 'sent' : sent})

In [ ]:
# Forming dataset json
dataset = (pos_dataset + neg_dataset_1 + neg_dataset_2)
random.shuffle(dataset)
split_point = int(len(dataset) * 0.8)
final_dataset = {'train' : dataset[:split_point], 'valid' : dataset[split_point:]}
json.dump(final_dataset, open('data/my_dataset.json', 'w'))

In [ ]:
# Forming dataset in datasets format
temp_train = load_dataset('json', data_files='data/my_dataset.json', field='train')
temp_valid = load_dataset('json', data_files='data/my_dataset.json', field='valid')
temp_train['valid'] = temp_valid['train']
temp_train.save_to_disk('data/single-ollie')

<h3> Training test

In [ ]:
dataset = load_from_disk('data/single-ollie')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_special_tokens({'additional_special_tokens' : ['<RELATION>']})

In [ ]:
training_args = TrainingArguments("data/single-ollie", evaluation_strategy="epoch")

In [ ]:
training_args

In [ ]:
def preprocess_sf1(examples):
    # return BatchEncoding(tokenizer(examples['ent1'], examples['ent2'], padding=True, truncation=True, max_length=100, return_tensors="pt"))
    query = ['%s <RELATION> %s' % (ent1, ent2) for ent1, ent2 in zip(examples['ent1'], examples['ent2'])]
    return tokenizer(query, examples["sent"], padding="max_length", truncation=True)
    
train_dataset = dataset['train'].map(preprocess_sf1, batched=True)
valid_dataset = dataset['valid'].map(preprocess_sf1, batched=True)

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=valid_dataset)

In [ ]:
trainer.train()